In [31]:
import math

In [32]:
def init_nonsmooth(dx):
    M = int(math.pi / dx)
    U0 = []
    for j in range(0, M):
        if j <= M / 2:
            U0.append(j * math.pi / M)
        else:
            U0.append(math.pi - j * math.pi / M)
    U0.append(0)
    return U0


def init_smooth(dx):
    M = int(math.pi / dx)
    U0 = []
    for j in range(0, M):
        x = j * math.pi / M
        U0.append(1 - math.exp(- math.sin(x) ** 2))
    U0.append(0)
    return U0

In [33]:
def fwd_solver(dt, dx, end_time, U0):

    U_list = [U0]
    T = int(end_time / dt)
    for n in range(0, T):
        new_U = update_temperature(U_list[-1], dt, dx)
        U_list.append(new_U)

    return U_list[-1]


def update_temperature(current_temp, dt, dx):
    new_temp = []
    M = len(current_temp)
    for j in range(0, M):

        if j == 0 or j == M - 1:
            new_temp_j = 0
        else:
            new_temp_j = current_temp[j] + ((sigma * dt) / (dx ** 2)) * (current_temp[j - 1] -
                                                    2 * current_temp[j] + current_temp[j + 1])

        new_temp.append(new_temp_j)

    return new_temp

In [34]:
alpha = 1 / 2
sigma = 1

fixed_time = 10  # in seconds

wh_list = []

dx = math.pi / 100

U0 = init_smooth(dx)
for h in [1, 2, 4]:
    dt = (dx ** 2 * alpha / h) / sigma

    curr_temp = fwd_solver(dt, dx, fixed_time, U0)
    rode_middle_temp_h = curr_temp[int(len(curr_temp) / 2)]
    wh_list.append(rode_middle_temp_h)

q = (wh_list[0] - wh_list[1]) / (wh_list[1] - wh_list[2])

print(f"beta: {math.log2(abs(q))}")

beta: 0.9989733635769921


In [35]:
### Q2

In [36]:
import numpy as np

def TDMSolver(a, b, c, u):
    N = len(a)

    ac = np.array(a)
    bc = np.array(b)
    cc = np.array(c)
    uc = np.array(u)

    for i in range(1, N):
        w = ac[i] / bc[i - 1]
        bc[i] = bc[i] - w * cc[i - 1]
        uc[i] = uc[i] - w * uc[i - 1]

    x = np.zeros((N,))
    x[N - 1] = uc[N - 1] / bc[N - 1]

    for i in range(N - 2, -1, -1):
        x[i] = (uc[i] - cc[i] * x[i + 1]) / bc[i]

    return x

In [37]:
def solver(dt, dx, end_time, U0):
    T = int(end_time / dt)

    alpha = (sigma * dt) / (dx ** 2)

    # Tridiagonal solver

    a = np.zeros((len(U0), ))
    b = np.zeros((len(U0), ))
    c = np.zeros((len(U0), ))

    for i in range(0, len(U0)):
        if i == 0 or i == len(U0) - 1:
            b[i] = 1
            a[i] = 0
            c[i] = 0

        else:

            b[i] = 1 + alpha
            c[i] = -alpha/2
            a[i] = -alpha/2

    U = U0
    U_checkpoints = [U0]
    for t in range(1, T + 1):
        R = []
        for j in range(len(U)):
            if j == 0:
                Rj = 0
            elif j == len(U) - 1:
                Rj = 0
            else:
                Rj = alpha/2 * U[j-1] + (1 - alpha) * U[j] + alpha/2 * U[j+1]

            R.append(Rj)

        new_U = TDMSolver(a, b, c, R)
        U_checkpoints.append(new_U)
        U = new_U

    return U_checkpoints[-1]

In [38]:
sigma = 1
dx = math.pi / 500
fixed_time = 8 # in seconds

# initial function
U0 = init_smooth(dx)
mid_temps = []

for h in [1, 2, 4]:
    dt = dx/h

    curr_temp = solver(dt, dx, fixed_time, U0)

    mid_temp = curr_temp[int(len(curr_temp) / 2)]
    mid_temps.append(mid_temp)

q = (mid_temps[0] - mid_temps[1]) / (mid_temps[1] - mid_temps[2])
print(math.log2(abs(q)))

1.99999179764568
